(1) Import libararies

In [1]:
import cv2
import glob
import dlib
import numpy as np
import scipy.io as sio
from keras import backend as BK
from keras.models import load_model
import os

Using TensorFlow backend.
C:\Users\sally\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sally\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\sally\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\sally\AppData\Roa

In [2]:
speaker = 's9'

gabor_path = 'D:/Mrs_backup/speech_test/gabor_feature/'+ speaker + '/'
audio_path = 'D:/Mrs_backup/speech_test/AudSpecs/'+ speaker + '/'
autoencoder_path= 'D:/Mrs_backup/speech_test/autoencoder/'+ speaker + '_trainModel/'
integrate_data_path ='D:/Mrs_backup/speech_test/model_input/gf/'+ speaker + '/'
if not os.path.exists(integrate_data_path):
    os.mkdir(integrate_data_path)
    
print('gabor input path is '+ gabor_path)
print('audio path is '+ audio_path)
print('autoencoder path is: '+autoencoder_path)
print('integrate path is: '+integrate_data_path)

num_files = len(os.listdir(gabor_path))
print(num_files)
num_slices=15
slice_length=26
num_features=7
num_frame =num_slices*slice_length
time_diff =3

gabor input path is D:/Mrs_backup/speech_test/gabor_feature/s9/
audio path is D:/Mrs_backup/speech_test/AudSpecs/s9/
autoencoder path is: D:/Mrs_backup/speech_test/autoencoder/s9_trainModel/
integrate path is: D:/Mrs_backup/speech_test/model_input/gf/s9/
100


In [3]:
# *****************************  Interplate the length of frame equals to the video length *************************#
import numpy as np
from scipy.ndimage.interpolation import map_coordinates

def interp_Frame(A):
    #print("A.shape IS ",A.shape)
    
    new_dims = []
    for original_length, new_length in zip(A.shape, (num_frame,num_features)):
        new_dims.append(np.linspace(0, original_length-1, new_length))

    coords = np.meshgrid(*new_dims, indexing='ij')
    B = map_coordinates(A, coords)
    #print('B.shape IS ',B.shape)
    return B

# *****************************  Define diff function *************************#
def diff(buf_input):
    buf_input=np.pad(buf_input,((1,0),(0,0)),'edge')
    buf_output=np.diff(buf_input,axis=0)
    return buf_output


# ********************** slice_video_3D function to divide into 15 non-overlap slice  **************************#
def slice_video_3D(video):
    video_output =np.empty((num_slices,slice_length,num_features*time_diff), np.dtype('float32'))
    start=0
    #print(video.shape)
    for i in range(0,num_slices):
        video_output[i,:,:]=video[start:start+slice_length,:]
        start+=slice_length
    return video_output

In [4]:
video_input =np.empty((num_files*(num_slices),int(slice_length),int(num_features)*time_diff), np.dtype('float32'))
print(video_input.shape)

(1500, 26, 21)


In [5]:
j=0
for filename in os.listdir(gabor_path): 
    src =gabor_path+ filename 
    print(src)
    format_num1="{number:03}".format(number=j)
    dst =gabor_path + str(format_num1)+'.mat'
    print(dst)
    os.rename(src, dst)
    j=j+1   

D:/Mrs_backup/speech_test/gabor_feature/s9/bbae4n.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/000.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bbae5s.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/001.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bbae6p.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/002.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bbae7a.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/003.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bbak8n.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/004.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bbak9s.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/005.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bbal1a.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/006.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bbalzp.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/007.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bbar2n.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/008.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bbar3s.mat
D

D:/Mrs_backup/speech_test/gabor_feature/s9/bgbg9a.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/085.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bgbn1s.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/086.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bgbn2p.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/087.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bgbn3a.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/088.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bgbnzn.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/089.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bgbt4n.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/090.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bgbt5s.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/091.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bgbt6p.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/092.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bgbt7a.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/093.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/bgif8n.mat
D

In [6]:
start = 0
for i in range(num_files):
#for i in range(1):

    #------------------- Get data path ------------------------------ *
    format_i="{number:03}".format(number=i)
    data_path = gabor_path + format_i+ '.mat'
    print(data_path)
    
    #------------------- Load data from data path  ------------------------------ *
    gabor_data = sio.loadmat(data_path)
    gabor_data = gabor_data['gabor_input']
    
    
    #------------- Call interplation function to interplate the length of frame equals to video length ------------ *
    gabor_data = interp_Frame(gabor_data)


     # ---------------------  Call diff function --------------------------------*
    diff_video=np.empty((gabor_data.shape[0],num_features*time_diff))
    diff_video[:,0:num_features*1]=gabor_data
    diff_video[:,num_features:num_features*2]=diff(gabor_data)
    diff_video[:,num_features*2:num_features*3] =diff(diff_video[:,num_features:num_features*2])
    #print(diff_video.shape)
    
    #------------------------- Call slice_video_3D function ----------------------*
    data_vid=slice_video_3D(diff_video)
    
    # --------------------- Add total number of slices ---------------------------*
    video_input[start:start+num_slices,:,:]=data_vid
    start+=num_slices
    
    
print('Video slices shape:'+str(video_input.shape))

D:/Mrs_backup/speech_test/gabor_feature/s9/000.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/001.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/002.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/003.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/004.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/005.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/006.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/007.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/008.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/009.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/010.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/011.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/012.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/013.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/014.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/015.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/016.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/017.mat
D:/Mrs_backup/speech_test/gabor_feature/s9/018.mat
D:/Mrs_backup/speech_test/gabor

In [7]:
print(video_input.shape)
print(video_input[200,0:3,:])
print("------------------")

(1500, 26, 21)
[[ 3.81430130e+01  5.04690552e+00  1.36511566e+02  3.40679893e+01
   1.01390619e+01  7.51926498e+01  2.16581151e-02 -1.85427451e+00
   1.65211521e-02 -1.46633711e+01  4.62911844e-01 -2.92353392e-01
   9.76910305e+00 -1.33774546e-03 -1.86584544e+00 -5.16967028e-02
  -1.48838453e+01  4.00503695e-01 -3.99809480e-01  8.65087700e+00
  -1.23356807e-03]
 [ 3.14966888e+01  2.22970915e+00  6.08291702e+01  3.30743713e+01
   1.16558695e+00  4.89932442e+01  9.32986848e-03 -6.64632559e+00
  -2.81719637e+00 -7.56824036e+01 -9.93616402e-01 -8.97347450e+00
  -2.61994057e+01 -1.23282457e-02 -4.79205132e+00 -2.83371758e+00
  -6.10190315e+01 -1.45652819e+00 -8.68112183e+00 -3.59685097e+01
  -1.09904995e-02]
 [ 3.07293587e+01  1.87635374e+00  5.17873001e+01  3.29297104e+01
   2.42499672e-02  4.52281532e+01  7.73103302e-03 -7.67329276e-01
  -3.53355497e-01 -9.04186726e+00 -1.44662410e-01 -1.14133692e+00
  -3.76508927e+00 -1.59883581e-03  5.87899637e+00  2.46384096e+00
   6.66405334e+01  8.48

(4) Audio features

In [8]:
# Define cost function [mean squared error + correlation loss]
def corr2_mse_loss(a,b):
    a = BK.tf.subtract(a, BK.tf.reduce_mean(a))
    b = BK.tf.subtract(b, BK.tf.reduce_mean(b))
    tmp1 = BK.tf.reduce_sum(BK.tf.multiply(a,a))
    tmp2 = BK.tf.reduce_sum(BK.tf.multiply(b,b))
    tmp3 = BK.tf.sqrt(BK.tf.multiply(tmp1,tmp2))
    tmp4 = BK.tf.reduce_sum(BK.tf.multiply(a,b))
    r = -BK.tf.divide(tmp4,tmp3)
    m=BK.tf.reduce_mean(BK.tf.square(BK.tf.subtract(a, b)))
    rm=BK.tf.add(r,m)
    return rm

# Define slice_audio_spec function to to divide into 15 non-overlap slices each of length 26
def slice_audio_spec(audio_spec):
    global AUDIO_LENGTH
    window_size=int(AUDIO_LENGTH/num_slices) #from time to number of audio index 
    audio_output =np.empty((num_slices,audio_spec.shape[0],window_size), np.dtype('float32'))
    start=0
    for i in range(0,num_slices):
        audio_output[i,:,:]=audio_spec[:,start:start+window_size]
        start+=window_size
        if start>AUDIO_LENGTH-window_size:
            break
    return audio_output

# Define get activations function # Extract the 32-bin bottleneck features as target for the main network
def get_activations(model, layer_in, layer_out, X_batch):
    get_activations = BK.function([model.layers[layer_in].input, BK.learning_phase()], [model.layers[layer_out].output])
    activations = get_activations([X_batch,0])
    return activations

#Define padding function
def get_padded_spec(data): 
    # Compress the spectrogram by raising to the power 1/3
    data=np.power(data,.3)
    # Get the video length
    t=data.shape[1]
    # Get the number of pads
    num_pads=int((2*num_slices)-(t%(2*num_slices)))
    # Add padding to the video length
    padded_data=np.pad(data,((0,0),(0,num_pads)),'reflect')
    #print(padded_data.shape)
    return padded_data

In [9]:
# Load autoencoder model
print('Loading autoencoder model...')
config = BK.tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = BK.tf.Session(config=config)
model=load_model(autoencoder_path+'autoencoder.h5',custom_objects={'corr2_mse_loss': corr2_mse_loss})
model.load_weights(autoencoder_path+'autoencoder_weights.h5')

Loading autoencoder model...


In [10]:
j=0
for filename in os.listdir(audio_path): 
    src =audio_path+ filename 
    print(src)
    format_num1="{number:03}".format(number=j)
    dst =audio_path + str(format_num1)+'.mat'
    print(dst)
    os.rename(src, dst)
    j=j+1   

D:/Mrs_backup/speech_test/AudSpecs/s9/bbae4n.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/000.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bbae5s.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/001.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bbae6p.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/002.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bbae7a.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/003.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bbak8n.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/004.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bbak9s.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/005.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bbal1a.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/006.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bbalzp.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/007.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bbar2n.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/008.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bbar3s.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/009.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bbar4p.mat
D

D:/Mrs_backup/speech_test/AudSpecs/s9/bgbn1s.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/086.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bgbn2p.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/087.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bgbn3a.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/088.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bgbnzn.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/089.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bgbt4n.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/090.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bgbt5s.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/091.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bgbt6p.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/092.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bgbt7a.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/093.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bgif8n.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/094.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bgif9s.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/095.mat
D:/Mrs_backup/speech_test/AudSpecs/s9/bgig1a.mat
D

In [11]:
import fnmatch
audioSpec = 'D:/Mrs_backup/speech_test/AudSpecs/'
audio_path = 'D:/Mrs_backup/speech_test/AudSpecs/'+ speaker + '/'

#Create a text file for audio input path
file = open(audioSpec +'s9_valid_aud_specs.txt','w')   
file.close()

# Get the amount of files in face folder [total 10 videos in the folders]
numfiles=len(fnmatch.filter(os.listdir(audio_path), '*.mat'))
print(numfiles)

for j in range(0,numfiles):
    format_num1="{number:03}".format(number=j)
    #  Write path to audio input file
    with open(audioSpec +'s9_valid_aud_specs.txt', 'a') as file:
        file.write(audio_path + str(format_num1)+'.mat\n')

100


In [12]:

# Open the text file of auditory spectrogram input path 
text_file = open(audioSpec+'s9_valid_aud_specs.txt','r')

# Load each line of the text file
lines = text_file.read().split('\n')
index_shuf=list(range(len(lines)))
lines_shuf=[]
for i in index_shuf:
    lines_shuf.append(lines[i])

# Get the number of audios
num_audios=len(lines)
print(num_audios)

# Get data shape 
mat=sio.loadmat(lines[0])
data = mat['y'].T[:,2:]

# get_padded_spec function to get the shape after padding data
padded_data=get_padded_spec(data=data)
global AUDIO_LENGTH
AUDIO_LENGTH=padded_data.shape[1]
# global AUDIO_LENGTH
# AUDIO_LENGTH= 182
#print(AUDIO_LENGTH)

# Call get_activations function to get the shape after bottleneck features
bottleneck=get_activations(model, 0, 12, padded_data.T)
#bottleneck=get_activations(model, 0, 15, padded_data.T)
#bottleneck=get_activations(model, 0, 18, padded_data.T)
bottleneck=bottleneck[0].T

# Get the total shape of audio_input variable
audio_input =np.empty((num_audios*(num_slices),bottleneck.shape[0],int(AUDIO_LENGTH/num_slices)), np.dtype('float32'))
print("Audio slices shape:" + str(audio_input.shape))

tmp =np.zeros((AUDIO_LENGTH), np.dtype('float32'))


i=0
for row in lines_shuf:
    
    # Load data from the path
    mat=sio.loadmat(row)
    
    # Read data from the second feature
    data = mat['y'].T[:,2:]
    
    # Call get_padded_spec function: 
    # (1). Compress the spectrogram by raising to the power 1/3 
    # (2). Add padding to the video length 
    padded_data=get_padded_spec(data=data)
    #print(padded_data.shape)
    
    #padded_data=interp_Spectrog(padded_data) 
    #print(padded_data)
    # Call get_activations function to get the bottleneck feature from autoencoder model 
    # Encoder auditory spectrogram
    bottleneck=get_activations(model, 0, 12, padded_data.T)
    #bottleneck=get_activations(model, 0, 15, padded_data.T)
    #bottleneck=get_activations(model, 0, 18, padded_data.T)
    
    #Transpose bottleneck[0] varaible
    bottleneck=bottleneck[0].T
       
    # Call slice_audio_spec function to divide into 15 non-overlap audio slices each of length [390/15] 26
    data=slice_audio_spec(bottleneck)
    #print(data.shape)
    
    #Get the total audio slices
    audio_input[i*(num_slices):(i+1)*(num_slices),:,:]=data[:,:,:]
    i+=1
    if i>=num_audios:
        break
    if i%10==0:
        print(str(i)+'/'+str(num_audios))

audio_output=np.reshape(audio_input,(audio_input.shape[0],audio_input.shape[1]*audio_input.shape[2]))
print('Audio slices shape:'+str(audio_input.shape))
print('Target features to network shape:'+str(audio_output.shape))

100
Audio slices shape:(1500, 32, 26)
10/100
20/100
30/100
40/100
50/100
60/100
70/100
80/100
90/100
Audio slices shape:(1500, 32, 26)
Target features to network shape:(1500, 832)


In [13]:
print(audio_input[500,:,0])
print(audio_output[500,])

[0.85588515 0.7920484  0.13406023 0.18011783 0.8735756  0.30450106
 0.83890486 0.48971727 0.45844057 0.602279   0.3363721  0.45479774
 0.14442568 0.6950786  0.12867342 0.7505889  0.57465464 0.8781084
 0.10916785 0.7941569  0.07842095 0.48782843 0.6441505  0.9292123
 0.0850739  0.08650184 0.54891145 0.7395258  0.17210689 0.35343462
 0.36369368 0.4723334 ]
[0.85588515 0.8711101  0.7423044  0.49327436 0.2748552  0.19673684
 0.185646   0.25803685 0.33227235 0.36501202 0.4459705  0.5012536
 0.5189434  0.52829254 0.51406735 0.47158432 0.45359513 0.46476075
 0.34528786 0.42215693 0.3275353  0.411104   0.3883824  0.27003077
 0.36675924 0.20494656 0.7920484  0.79041827 0.773759   0.80740184
 0.8227801  0.8549583  0.8903453  0.78266233 0.75352335 0.7122332
 0.75481296 0.8077307  0.8301641  0.8240601  0.86309546 0.9228153
 0.9309842  0.879362   0.7624031  0.65964234 0.66598797 0.662885
 0.68845457 0.7164571  0.532544   0.39258328 0.13406023 0.14546426
 0.19883716 0.25727656 0.22215918 0.2241046  

In [14]:
print(video_input.shape[1])
print(audio_input.shape)

26
(1500, 32, 26)


In [15]:
N=80
num_test=20
num_train=num_audios-num_test
L=int(np.ceil(num_train/N)*num_slices)
i=0
for i in range(N):
    if i<79:
        print('Saving data part'+str(i+1)+'...')
        print('Saving test data')
        start=i*L
        end=(i+1)*L
        print(str(start)+' to '+str(end))
        sio.savemat(integrate_data_path+'preprocessed_data_final_part'+str(i+1)+'.mat', mdict={'video_input': video_input[start:end,:,:], 'audio_input' : audio_input[start:end,:,:]})
    else:
        print('Saving data part'+str(i+1)+'...')
        start=i*L
        end=num_train*num_slices
        print(str(start)+' to '+str(end))
        sio.savemat(integrate_data_path+'preprocessed_data_final_part'+str(i+1)+'.mat', mdict={'video_input': video_input[start:end,:,:], 'audio_input' : audio_input[start:end,:,:]})

print('Saving validation data...')
start=num_train*num_slices
print(str(start)+' to '+str(video_input.shape[0]))
sio.savemat(integrate_data_path+'preprocessed_data_final_validation.mat', mdict={'video_input': video_input[start:,:,:], 'audio_input' : audio_input[start:,:,:]})

Saving data part1...
Saving test data
0 to 15
Saving data part2...
Saving test data
15 to 30
Saving data part3...
Saving test data
30 to 45
Saving data part4...
Saving test data
45 to 60
Saving data part5...
Saving test data
60 to 75
Saving data part6...
Saving test data
75 to 90
Saving data part7...
Saving test data
90 to 105
Saving data part8...
Saving test data
105 to 120
Saving data part9...
Saving test data
120 to 135
Saving data part10...
Saving test data
135 to 150
Saving data part11...
Saving test data
150 to 165
Saving data part12...
Saving test data
165 to 180
Saving data part13...
Saving test data
180 to 195
Saving data part14...
Saving test data
195 to 210
Saving data part15...
Saving test data
210 to 225
Saving data part16...
Saving test data
225 to 240
Saving data part17...
Saving test data
240 to 255
Saving data part18...
Saving test data
255 to 270
Saving data part19...
Saving test data
270 to 285
Saving data part20...
Saving test data
285 to 300
Saving data part21...
S